# Fusion Localization Visualization

融合定位中间变量可视化

In [ ]:
import re
from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np

## Data Loading

In [ ]:
log_file_path = "./tracking_control_node-2-stdout.log"

TUPLE_PATTERN = re.compile(
    r"(?P<key>state|z) = \((?P<value>[^)]+)\)"
)
FLOAT_PATTERNS = (
    re.compile(r"(?P<key>gnss_X)=(?P<value>-?\d*(?:\.\d*)?)"),
    re.compile(r"(?P<key>gnss_Y)=(?P<value>-?\d*(?:\.\d*)?)"),
    re.compile(r"(?P<key>enu_x)=(?P<value>-?\d*(?:\.\d*)?)"),
    re.compile(r"(?P<key>enu_y)=(?P<value>-?\d*(?:\.\d*)?)"),
    re.compile(r"(?P<key>enu_z)=(?P<value>-?\d*(?:\.\d*)?)"),
)

data_dict = defaultdict(list)
with open(log_file_path, "r", encoding="utf-8") as log_file:
    for line in log_file:
        if match_result := TUPLE_PATTERN.search(line):
            key = match_result.group("key")
            value = tuple(
                float(s.strip())
                for s in match_result.group("value").split(',')
            )
            data_dict[key].append(value)
        for pattern in FLOAT_PATTERNS:
            if match_result := pattern.search(line):
                key = match_result.group("key")
                value = float(match_result.group("value"))
                data_dict[key].append(value)

{ key: len(value) for key, value in data_dict.items() }

In [ ]:
data_list = [
    {key: data_dict[key][i] for key in data_dict.keys()}
    for i in range(len(data_dict["state"]))
]
heading = np.array([item["state"][2] for item in data_list])

transponder_correction_filter = np.array([
    (item["z"][0] != item["state"][0]) or (item["z"][1] != item["state"][1])
    for item in data_list
])
print(
    "%d out of %d records are corrected by transponders." % (
        transponder_correction_filter.sum(),
        len(transponder_correction_filter)
    )
)

## ENU

In [ ]:
x_enu = np.array(data_dict["enu_x"])
y_enu = np.array(data_dict["enu_y"])
z_enu = np.array(data_dict["enu_z"])

fig = plt.figure(figsize=(6, 6), dpi=150)
fig.set_facecolor("#fff")
ax = fig.add_subplot()

ax.plot(x_enu, y_enu, "b.-", ms=2, alpha=0.5, label="ENU")
ax.plot(x_enu[0], y_enu[0], "ro", ms=8, alpha=0.5, label="ENU Start")

ax.set(
    xlabel="x",
    xlim=(x_enu.min() - 5, x_enu.max() + 5),
    ylabel="y",
    ylim=(y_enu.min() - 5, y_enu.max() + 5),
    aspect="equal",
)
ax.legend()
ax.grid(alpha=0.3)

## Center Position By GNSS

In [ ]:
x_gnss = np.array(data_dict["gnss_X"])
y_gnss = np.array(data_dict["gnss_Y"])

fig = plt.figure(figsize=(6, 6), dpi=150)
fig.set_facecolor("#fff")
ax = fig.add_subplot()

ax.plot(x_gnss, y_gnss, "b.-", ms=2, alpha=0.5, label="GNSS")
ax.plot(x_gnss[0], y_gnss[0], "ro", ms=8, alpha=0.5, label="GNSS Start")

ax.set(
    xlabel="x",
    xlim=(x_gnss.min() - 5, x_gnss.max() + 5),
    ylabel="y",
    ylim=(y_gnss.min() - 5, y_gnss.max() + 5),
    aspect="equal",
)
ax.legend()
ax.grid(alpha=0.3)

## Recomputed Center Position By GNSS

In [ ]:
INS_OFFSET_X = 0.313919334029
INS_OFFSET_Y = 2.814034003987

cos_heading = np.cos(heading)
sin_heading = np.sin(heading)
x_recomputed = 605.027583807785 \
    + INS_OFFSET_X * -cos_heading \
    + INS_OFFSET_Y * sin_heading \
    + 0.758406531503 * x_enu \
    + 0.582665705457 * y_enu \
    + -0.422567712749 * z_enu \
    + 0.000369509915 * (x_enu * y_enu) \
    + -0.002675677792 * (y_enu * z_enu) \
    + 0.008348928616 * (z_enu * x_enu) \
    + -0.001408113454 * pow(x_enu, 2) \
    + 0.000944304804 * pow(y_enu, 2) \
    + -0.062587808085 * pow(z_enu, 2)
y_recomputed = 86.897404417344 \
    + INS_OFFSET_X * -sin_heading \
    + INS_OFFSET_Y * -cos_heading \
    + -0.630523471739 * x_enu \
    + 0.772013632684 * y_enu \
    + -0.576925384072 * z_enu \
    + -0.000238724420 * (x_enu * y_enu) \
    + 0.000733593895 * (y_enu * z_enu) \
    + -0.001219673340 * (z_enu * x_enu) \
    + -0.000961266593 * pow(x_enu, 2) \
    + 0.000395562848 * pow(y_enu, 2) \
    + -0.056915073525 * pow(z_enu, 2)

fig = plt.figure(figsize=(6, 6), dpi=150)
fig.set_facecolor("#fff")
ax = fig.add_subplot()

ax.plot(x_recomputed, y_recomputed, "b.-", ms=2, alpha=0.5, label="Recomputed")
ax.plot(x_recomputed[0], y_recomputed[0], "ro", ms=8, alpha=0.5, label="Recomputed Start")

ax.set(
    xlabel="x",
    xlim=(x_recomputed.min() - 5, x_recomputed.max() + 5),
    ylabel="y",
    ylim=(y_recomputed.min() - 5, y_recomputed.max() + 5),
    aspect="equal",
)
ax.legend()
ax.grid(alpha=0.3)